Testing effective connectivity from time series prediction using foundation models. The initial case is without fine tuning

In [ ]:
pip install timesfm sktime

In [ ]:
import matplotlib.pyplot as plt
import timesfm
import pandas as pd
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error, mean_absolute_scaled_error, mean_absolute_error
from sktime.forecasting.naive import NaiveForecaster
from sktime.forecasting.base import ForecastingHorizon
from sklearn.linear_model import LinearRegression
from sktime.forecasting.compose import make_reduction
from sktime.forecasting.statsforecast import StatsForecastAutoARIMA, StatsForecastAutoETS
import os
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np
from scipy.stats import f, pearsonr, ttest_ind
from sklearn.linear_model import Ridge, LinearRegression
import warnings
warnings.filterwarnings("ignore", message="possible convergence problem")


In [ ]:
##Initialize foundation model without fine tuning
HORIZON = 1
tfm = timesfm.TimesFm(
      hparams=timesfm.TimesFmHparams(
          backend="gpu",
          per_core_batch_size=32,
          horizon_len=HORIZON,
      ),
      checkpoint=timesfm.TimesFmCheckpoint(
          huggingface_repo_id="google/timesfm-1.0-200m-pytorch"),
  )

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
##Load time series
def load_data(path, verbose = False):
    dataset = pd.read_csv(path, sep='\t', header = None)
    if verbose:
        print(dataset.columns)
        print(len(dataset))
        print(dataset.head())
    return dataset
control_data = load_data('sub-CON001_ses-control_task-rest_space-MNI152NLin2009cAsym_atlas-Schaefer117_timeseries.tsv')

FileNotFoundError: [Errno 2] No such file or directory: 'sub-CON001_ses-control_task-rest_space-MNI152NLin2009cAsym_atlas-Schaefer117_timeseries.tsv'

In [ ]:
# convert the structure to be read by the model
def split_train_test(data, break_index):
    split = data.index > HORIZON -1
    return data[split], data[~split]
control_data_train, control_data_test = split_train_test(control_data, HORIZON)

def convert_to_timefm(data):
    data_time_fm = []
    for col in data.columns:
        data_time_fm += [data[col]]
    return data_time_fm
control_data_train_for_time_fm = convert_to_timefm(control_data_train)
print(np.shape(control_data_train))
print(np.shape(control_data_test))

(933, 3)
(100, 3)


In [ ]:
# Individual prediction
# freq=[0] means no seasonality
predicted = tfm.forecast(control_data_train_for_time_fm, freq=[0] * len(control_data_train.columns))[0]
#predicted = tfm.forecast(control_data_train_for_time_fm, freq=[0] * len(control_data_train_for_time_fm.columns))[0] #adding this will check only one series [0]

In [ ]:
#  Granger-like Causality Methods Using against Granger


# Generate test data (your logistic map coupling)
def generate_test_data_LM():
    n = 100
    r = 3.9
    coupling_strength = 0.3

    X = np.zeros((3, n))
    X[0, 0] = 0.1 + np.random.uniform(-0.01, 0.01)
    X[1, 0] = 0.2 + np.random.uniform(-0.01, 0.01)
    X[2, 0] = 0.3 + np.random.uniform(-0.01, 0.01)

    for i in range(1, n):
        X[0, i] = r * X[0, i-1] * (1 - X[0, i-1])
        X[1, i] = r * X[1, i-1] * (1 - X[1, i-1])
        X[2, i] = r * X[2, i-1] * (1 - X[2, i-1])

        X[1, i] += coupling_strength * X[0, i-1]  # x -> y
        X[2, i] += coupling_strength * X[1, i-1]  # y -> z

        X[:, i] = np.clip(X[:, i], 0, 1)

    return pd.DataFrame(X.T, columns=['x', 'y', 'z'])


def generate_test_data_OU():
    """
    Generates test data using an Ornstein-Uhlenbeck process with causal structure.

    Returns:
        pd.DataFrame: Standardized time series data with columns ['X0', 'X1', 'X2']
                      where:
                      - X1 is influenced by X0
                      - X2 is influenced by X1
    """
    n = 200  # Number of time steps
    dt = 0.01  # Time step size

    # Coupling matrix defining causal relationships
    theta = np.array([[1.0, -8.0, 0.0],   # X0 is influenced by X1
                      [0.0, 1.0, 0.0],  # X1 is influenced by X2 (strong negative)
                      [0.0, 0.0, 1.0]]) # X2 is influenced by X0 (strong negative)

    sigma = 0.01  # Noise level

    # Initialize time series
    X = np.zeros((n, 3))

    # Generate OU process with Euler-Maruyama method
    for t in range(1, n):
        X[t] = X[t-1] + dt * (-theta @ X[t-1]) + np.sqrt(dt) * sigma * np.random.randn(3)

    # Standardize the data
    means = X.mean(axis=0)
    stds = X.std(axis=0)
    stds[stds == 0] = 1.0  # Prevent division by zero
    X_standardized = (X - means) / stds

    return pd.DataFrame(X, columns=['x', 'y', 'z'])



# Dummy TimesFM function (replace with your actual tfm)
def timesfm_forecast(series_data, horizon):
    """Replace this with your actual tfm.forecast call"""
    if hasattr(series_data, 'values'):
        data = series_data.values.flatten()
    else:
        data = np.array(series_data).flatten()

    if len(data) > 1:
        """
        # Dummy forecast - to replace with tfm.forecast([data], freq=[0])[0]
        # For now, simple AR(1) as placeholder
        rho = np.corrcoef(data[:-1], data[1:])[0, 1] if np.std(data) > 0 else 0
        rho = np.clip(rho, -0.99, 0.99)
        last_val = data[-1]
        mean_val = np.mean(data)
        """
        forecast = np.zeros(horizon)
        for i in range(horizon):
            #forecast[i] = mean_val + rho**i * (last_val - mean_val)

            forecast[i] = np.squeeze(tfm.forecast([data], freq=[0])[0])
            #print(forecast[i])

    else:
        forecast = np.full(horizon, np.mean(data))

    return forecast


# ================================
# METHOD : TimesFM Residual Analysis
# ================================
def timesfm_residual_causality(target_series, covariate_series, max_lag=3):
    """
    Use TimesFM to get baseline predictions, then test if covariate information
    explains the residuals
    """
    print("Method: TimesFM Residual Analysis")

    target_data = np.array(target_series).flatten()
    cov_data = np.array(covariate_series).flatten()

    min_len = min(len(target_data), len(cov_data))
    target_data = target_data[:min_len-1]  # -1 for prediction
    cov_data = cov_data[:min_len-1]

    results = []

    # Get TimesFM predictions for the entire series
    try:
        # Rolling predictions
        predictions = []
        actuals = []

        window_size = 30
        for i in range(window_size, len(target_data)):
            train_window = target_data[i-window_size:i]
            pred = timesfm_forecast(train_window, HORIZON)
            predictions.append(pred[0])
            actuals.append(target_data[i])

        predictions = np.array(predictions)
        actuals = np.array(actuals)
        residuals = actuals - predictions

        # Test if lagged covariates explain residuals
        for lag in range(1, max_lag + 1):
            if len(residuals) < lag + 10:
                continue

            # Align residuals with lagged covariates
            residuals_aligned = residuals[lag:]
            cov_lagged = cov_data[window_size:window_size + len(residuals_aligned)]

            if len(cov_lagged) != len(residuals_aligned):
                min_len_align = min(len(cov_lagged), len(residuals_aligned))
                residuals_aligned = residuals_aligned[:min_len_align]
                cov_lagged = cov_lagged[:min_len_align]

            if len(residuals_aligned) < 10:
                continue

            # Test correlation between lagged covariate and residuals
            if np.std(residuals_aligned) > 0 and np.std(cov_lagged) > 0:
                correlation, p_val_corr = pearsonr(cov_lagged, residuals_aligned)

                # Also test with linear regression
                reg = LinearRegression().fit(cov_lagged.reshape(-1, 1), residuals_aligned)
                r2_score = reg.score(cov_lagged.reshape(-1, 1), residuals_aligned)

                results.append({
                    'lag': lag,
                    'correlation': correlation,
                    'correlation_p_value': p_val_corr,
                    'r2_residual_explained': r2_score,
                    'covariate_coeff': reg.coef_[0],
                    'n_samples': len(residuals_aligned)
                })

    except Exception as e:
        print(f"Residual analysis failed: {e}")

    return results

# ================================
# METHOD : Classical Granger Test (Baseline)
# ================================
def classical_granger_test(target_series, covariate_series, max_lag):
    """
    Classical Granger causality test for comparison
    """
    print("Method : Classical Granger Test (Baseline)")

    target_data = np.array(target_series).flatten()
    cov_data = np.array(covariate_series).flatten()

    min_len = min(len(target_data), len(cov_data))
    target_data = target_data[:min_len]
    cov_data = cov_data[:min_len]

    results = []

    for lag in range(1, max_lag + 1):
        if len(target_data) < lag + 20:
            continue

        # Prepare data
        y = target_data[lag:]

        # Restricted model: y_t = α + β₁*y_{t-1} + ... + βₖ*y_{t-k} + ε
        X_restricted = np.ones((len(y), 1))  # intercept
        for l in range(1, lag + 1):
            X_restricted = np.column_stack([X_restricted, target_data[lag-l:-l if l > 0 else len(target_data)]])

        # Full model: add lagged covariates
        X_full = X_restricted.copy()
        for l in range(1, lag + 1):
            X_full = np.column_stack([X_full, cov_data[lag-l:-l if l > 0 else len(cov_data)]])

        try:
            # Fit models
            reg_restricted = LinearRegression().fit(X_restricted, y)
            reg_full = LinearRegression().fit(X_full, y)

            # Calculate RSS
            pred_restricted = reg_restricted.predict(X_restricted)
            pred_full = reg_full.predict(X_full)

            rss_restricted = np.sum((y - pred_restricted) ** 2)
            rss_full = np.sum((y - pred_full) ** 2)

            # F-test
            n = len(y)
            k = lag  # number of additional parameters

            if rss_full > 0:
                f_stat = ((rss_restricted - rss_full) / k) / (rss_full / (n - 2*lag - 1))
                from scipy.stats import f
                p_val = 1 - f.cdf(f_stat, k, n - 2*lag - 1) if f_stat > 0 else 1.0
            else:
                f_stat = 0
                p_val = 1.0

            # Extract covariate coefficients
            cov_coeffs = reg_full.coef_[-lag:] if len(reg_full.coef_) >= lag else []

            results.append({
                'lag': lag,
                'f_statistic': f_stat,
                'p_value': p_val,
                'covariate_coeffs': cov_coeffs.tolist(),
                'rss_improvement': rss_restricted - rss_full,
                'n_samples': n
            })

        except Exception as e:
            print(f"  Classical test lag {lag} failed: {e}")

    return results

# ================================
# Main Execution
# ================================
print("=== Testing Multiple Granger-like Causality Methods with TimesFM ===\n")

# Generate test data
control_data = generate_test_data_OU() #Generater data with Logistic map causality
split_point = int(0.8 * len(control_data))
control_data_train = control_data.iloc[:split_point]
control_data_test = control_data.iloc[split_point:]

from statsmodels.tsa.api import VAR
model = VAR(control_data_train)
lag_order_results = model.select_order(maxlags=15)
lags = lag_order_results.aic  # or use bic or hqic
print(f"Optimal lag (AIC): {lags}")


# Test all methods for known relationships
#test_relationships = [('x', 'y'), ('y', 'z'), ('z', 'x')]  # Include a null case
test_relationships = [('x', 'y'), ('y','x') , ('y', 'z'),('z', 'y'), ('z', 'x'),('x', 'z')]

for covariate_name, target_name in test_relationships:
    print(f"\n{'='*60}")
    print(f"Testing: {covariate_name} -> {target_name}")
    print(f"{'='*60}")

    target_series = control_data_train[target_name]
    covariate_series = control_data_train[covariate_name]


    # Method : LLM Residual analysis
    try:
        results3 = timesfm_residual_causality(target_series, covariate_series,lags)
        if results3:
            best_result = min(results3, key=lambda x: x['correlation_p_value'])
            print(f"Method - Best result: Lag={best_result['lag']}, p={best_result['correlation_p_value']:.4f}, "
                  f"corr={best_result['correlation']:.4f}")
        else:
            print("Method LLM - No results")
    except Exception as e:
        print(f"Method  LLM failed: {e}")

    # Method : Classical Granger (baseline)
    try:
        results4 = classical_granger_test(target_series, covariate_series,lags)
        if results4:
            best_result = min(results4, key=lambda x: x['p_value'])
            print(f"Method Granger - Best result: Lag={best_result['lag']}, p={best_result['p_value']:.4f}, "
                  f"F={best_result['f_statistic']:.3f}")
        else:
            print("Method Granger - No results")
    except Exception as e:
        print(f"Method Granger failed: {e}")

print(f"{'='*60}")

=== Testing Multiple Granger-like Causality Methods with TimesFM ===

Optimal lag (AIC): 1

Testing: x -> y
Method: TimesFM Residual Analysis
Method - Best result: Lag=1, p=0.0120, corr=-0.2214
Method : Classical Granger Test (Baseline)
Method Granger - Best result: Lag=1, p=0.2592, F=1.282

Testing: y -> x
Method: TimesFM Residual Analysis
Method - Best result: Lag=1, p=0.0063, corr=0.2403
Method : Classical Granger Test (Baseline)
Method Granger - Best result: Lag=1, p=0.0001, F=16.132

Testing: y -> z
Method: TimesFM Residual Analysis
Method - Best result: Lag=1, p=0.1545, corr=-0.1266
Method : Classical Granger Test (Baseline)
Method Granger - Best result: Lag=1, p=0.2899, F=1.128

Testing: z -> y
Method: TimesFM Residual Analysis
Method - Best result: Lag=1, p=0.0489, corr=0.1745
Method : Classical Granger Test (Baseline)
Method Granger - Best result: Lag=1, p=0.9148, F=0.011

Testing: z -> x
Method: TimesFM Residual Analysis
Method - Best result: Lag=1, p=0.2966, corr=0.0930
Meth